In [94]:
import requests
import re
import time
import random
import json
import os
cong_key=''

In [34]:
cong={}
next='https://api.congress.gov/v3/member?'
while True:
    member_resp=requests.get(f'{next}&api_key={cong_key}')
    for m in member_resp.json()['members']:
        cong[re.sub(r'[^a-zA-Z ]','',m['name'].lower()).strip()]=m['bioguideId']
    if not member_resp.json().get('pagination'):
        break
    next=member_resp.json().get('pagination').get('next')
    time.sleep(random.random()*2)

MissingSchema: Invalid URL 'None&api_key=Jq9Uy2kgbNouvTJfoixzEYYQQJ88FAjM3D5HFPne': No scheme supplied. Perhaps you meant https://None&api_key=Jq9Uy2kgbNouvTJfoixzEYYQQJ88FAjM3D5HFPne?

In [29]:
with open('./opensecrets_info.json', 'r') as f:
    cong_info=json.load(f)

In [67]:
from string import ascii_lowercase as alc
mapping={}
manual=[]
for o_id,info in cong_info.items():
    name=re.sub(r'[^a-zA-Z ]','',info['name'].lower()).strip()
    f,*n=name.split()
    if len(n)==1:
        new=" ".join([n[-1],f])
    elif len(n)==2:
        new=" ".join([n[-1],f,n[0]])

    #guess cycle
    if not cong.get(new):
        new+='  '
        for let in alc:
            new=new[:-1]+let
            if cong.get(new):break
    mapping[o_id]={'id':cong.get(new),'name':info['name']}
    if not cong.get(new):
        manual.append(info['name'])

In [73]:
usable={k:v for k,v in mapping.items() if v['id']}
new_info={v['id']:cong_info[k]['industry'] for k,v in usable.items()}

In [93]:
bill_id=set()
bills=[]
for file in os.listdir('./votes'):
    with open(f'./votes/{file}') as f:
        for z in json.load(f):
            if z[0] not in bill_id:
                bill_id.add(z[0])
                bills.append(z)

In [109]:
new_info={k:v for k,v in new_info.items() if v}
with open('./gov_id_industry.json', 'w') as f:
    json.dump(new_info,f)

['HR1023',
 'To repeal section 134 of the Clean Air Act, relating to the greenhouse gas reduction fund.',
 'Environmental Protection',
 '<p>This bill repeals the Greenhouse Gas Reduction Fund. This fund provides financial and technical assistance to states, nonprofit organizations, and other eligible recipients to help communities carry out activities to reduce greenhouse gas emissions.</p>',
 {'A000370': 'No',
  'A000055': 'Aye',
  'A000371': 'No',
  'A000379': 'Aye',
  'A000372': 'Aye',
  'A000376': 'No',
  'A000380': 'No',
  'A000369': 'Aye',
  'A000377': 'Aye',
  'A000375': 'Aye',
  'A000148': 'No',
  'B001291': 'Aye',
  'B001298': 'Aye',
  'B001307': 'Aye',
  'B001306': 'Aye',
  'B001318': 'No',
  'B001299': 'Aye',
  'B001282': 'Aye',
  'B001300': 'No',
  'B001314': 'Aye',
  'B001281': 'No',
  'B000668': 'Aye',
  'B001287': 'No',
  'B001301': 'Aye',
  'B001292': 'No',
  'B000740': 'Aye',
  'B001302': 'Aye',
  'B001257': 'Not Voting',
  'B000490': 'No',
  'B001311': 'Aye',
  'B0005